In [1]:
import os
import glob
import numpy as np
import xarray as xr
import dask
import time
import itertools

import sys
p = os.path.abspath('/global/homes/q/qnicolas/')
if p not in sys.path:
    sys.path.append(p)

from tools.e5tools import *
xr.set_options(display_style='text') 
from orographicPrecipitation.observationsAndERA5.mountainUtils import BL_plevs
from orographicPrecipitation.precip_model_functions import humidsat,qsat

from mountainUtils import *

In [2]:
import dask
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status"
from dask.distributed import Client
client=Client()

/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37059 instead
  http_address["port"], self.http_server.port


In [10]:
MAINDIR = "/global/cscratch1/sd/qnicolas/regionsDataBig/"

Ghats      = MountainRangeCustom("ghats"      ,[64,82,7,24]   ,"Western Ghats"            ,70 ,[6,7,8]  ,[ 75  ,   9  ,  71.5, 20   ,2  ])
Myanmar    = MountainRangeCustom("myanmar"    ,[84,102,7,24]  ,"Myanmar"                  ,60 ,[6,7,8]  ,[ 98  ,  11  ,  90  , 21   ,4  ])
Vietnam    = MountainRangeCustom("vietnam"    ,[100,118,8,25] ,"Annamite range (Vietnam)" ,240,[10,11]  ,[108  ,  18  , 111  , 13   ,2  ])
Malaysia   = MountainRangeCustom("malaysia"   ,[96,114,-2,15] ,"Malaysia"                 ,225,[11,12]  ,[102.5,   8  , 105.5,  3   ,2  ])
Philippines= MountainRangeCustom("philippines",[117,135,5,22] ,"Philippines"              ,225,[11,12]  ,[123.5,  17  , 127  ,  9   ,2  ])
Newbritain = MountainRangeCustom("newbritain" ,[145,163,-17,0],"New Britain"              ,320,[6,7,8]  ,[154  , - 5.5, 150  , -8.5 ,2  ])
SMO        = MountainRangeCustom("smo"        ,[240,270,15,35],"Sierra Madre Occidental"  ,60 ,[7,8,9]  ,[254, 21, 249, 27.5, 3],path = "/global/cscratch1/sd/qnicolas/regionsData/")
Cascades   = MountainRangeCustom("cascades"   ,[222,240,43,60],"British Columbia cascades",45 ,[1,11,12],[235,47,227,54,3])

MRs = (Ghats, Myanmar    ,Vietnam    ,Malaysia   ,Philippines,Newbritain,Cascades )   #SMO,)#Cascades,)#


#for MR in MRs:
#    print(MR.name)
#    MR.set_4dvar('T','128_130_t')
#    MR.set_4dvar('U','128_131_u')
#    MR.set_4dvar('V','128_132_v')
#    MR.set_4dvar('Q','128_133_q')
#    MR.set_4dvar('W','128_135_w')

In [5]:
Ghats      .box_computations = Ghats      .box
Myanmar    .box_computations = Myanmar    .box
Vietnam    .box_computations = Vietnam    .box
Malaysia   .box_computations = Malaysia   .box
Philippines.box_computations = Philippines.box
Newbritain .box_computations = Newbritain .box
SMO .box_computations = SMO .box
Cascades .box_computations = Cascades .box

allyears=range(2001,2021)

In [6]:
def resample_daily(ds):
    """performs the same job as resample(time="1D") but keeps missing days out"""
    ds.coords['time'] = ds.time.dt.floor('1D')
    return ds.groupby('time').mean()

# Loading daily data into zarr groups

## ERA5 plevs

In [6]:
def createzarr_era5_pl(MR,m,varcode):
    name = MR.name
    box = MR.box_computations
    years=allyears
    for y in years:
        filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name)
        if os.path.isdir(filepath):
            print("Already exists! - %s"%filepath)
            continue
        print("Doesn't exist! - %s"%filepath)
        ti=time.time()
        var = xr.open_mfdataset(ERA5D_PATH+"e5.oper.an.pl/{}{:02}/e5.oper.an.pl.{}.*.nc".format(y,m,varcode))
        sel_box(var,box).to_zarr(filepath,mode="w")
        print("%is"%(time.time()-ti))
        

In [7]:
for MR in MRs:
    months=MR.months
    for varcode in ["128_060_pv","128_130_t","128_131_u","128_132_v","128_133_q","128_135_w"]:
        for m in months:
            %time createzarr_era5_pl(MR,m,varcode)

Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200101.cascades.zarr
110s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200201.cascades.zarr
106s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200301.cascades.zarr
113s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200401.cascades.zarr
108s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200501.cascades.zarr
113s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200601.cascades.zarr
112s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200701.cascades.zarr
118s
Doesn't exist! - /global/cscratch1/sd/qnicolas/regionsDataBig/e5.oper.an.pl.128_060_pv.ll025sc.200801.cascades.zarr
112s
Doesn't exist! - /global/cscratc

## ERA5 sfc - TRMM - GPM

In [8]:
def createzarr_era5_sfc(MR,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isdir(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.sfc/*/e5.oper.an.sfc.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        
def createzarr_trmm(MR,m):
    name=MR.name
    box=MR.box_computations
    years=range(2010,2014)
    var = xr.open_mfdataset([glob.glob(CMIP6_FOLDER+"obs4mip/NASA-GSFC/TRMM/observations/atmos/pr/3hr/NASA-GSFC/TRMM/*/*_{}{:02}*.nc".format(y,m))[0] for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    sel_box(var,box).to_zarr(MAINDIR + "trmm.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name),mode="w")

        

In [9]:
def flatten(t):
    return [item for sublist in t for item in sublist]
def createzarr_gpm(MR,m):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)
    if os.path.isdir(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob("/global/cscratch1/sd/pnicknis/gpm/daily/3B-DAY.MS.MRG.3IMERG.{}{:02}*-S000000-E235959.V06.nc4".format(y,m)) for y in years])
    var = var.rename({'lat':'latitude','lon':'longitude'})
    var = var.reindex(latitude=list(reversed(var.latitude)))
    var = var.assign_coords({'longitude': var.longitude%360})
    var = var.sortby('longitude')
    var['time'] = var.indexes['time'].to_datetimeindex()
    sel_box(var,box).to_zarr(filepath,mode="w")


In [10]:
def createzarr_era5_vinteg(name,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.vinteg/*/e5.oper.an.vinteg.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        

In [11]:
def createzarr_era5_vinteg(MR,m,varcode):
    name=MR.name
    box=MR.box_computations
    years=allyears
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)
    if os.path.isfile(filepath):
        print("Already exists! - %s"%filepath)
        return
    var = xr.open_mfdataset([glob.glob(ERA5D_PATH+"e5.oper.an.vinteg/*/e5.oper.an.vinteg.{}.*.{}{:02}0100_*.nc".format(varcode,y,m))[0] for y in years])
    sel_box(var,box).to_zarr(filepath,mode="w")
        

## To be run

In [12]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for MR in MRs:
        print(MR.name)
        months=MR.months
        for varcode in ['228_246_100u','228_247_100v']:#,
            print(varcode)
            for m in months:
                print(m)
                %time createzarr_era5_sfc(MR,m,varcode)
        for varcode in ['162_071_viwve','162_072_viwvn']:
            print(varcode)
            for m in months:
                print(m)
                %time createzarr_era5_vinteg(MR,m,varcode)
        for m in months:
            print(m)
            createzarr_gpm(MR,m)

cascades
228_246_100u
1
CPU times: user 2.06 s, sys: 1.28 s, total: 3.34 s
Wall time: 19.8 s
11
CPU times: user 1.89 s, sys: 1.11 s, total: 3 s
Wall time: 13.7 s
12
CPU times: user 1.83 s, sys: 998 ms, total: 2.83 s
Wall time: 12.7 s
228_247_100v
1
CPU times: user 1.81 s, sys: 1.04 s, total: 2.85 s
Wall time: 13.6 s
11
CPU times: user 1.79 s, sys: 1.03 s, total: 2.82 s
Wall time: 13 s
12
CPU times: user 1.76 s, sys: 1.02 s, total: 2.78 s
Wall time: 12.9 s
162_071_viwve
1
CPU times: user 1.62 s, sys: 943 ms, total: 2.57 s
Wall time: 13 s
11
CPU times: user 1.43 s, sys: 972 ms, total: 2.4 s
Wall time: 11.5 s
12
CPU times: user 1.32 s, sys: 988 ms, total: 2.31 s
Wall time: 11.3 s
162_072_viwvn
1
CPU times: user 1.46 s, sys: 808 ms, total: 2.27 s
Wall time: 11.8 s
11
CPU times: user 1.48 s, sys: 765 ms, total: 2.24 s
Wall time: 11.2 s
12
CPU times: user 1.48 s, sys: 846 ms, total: 2.33 s
Wall time: 11.3 s
1


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()


11
12


/global/homes/q/qnicolas/.conda/envs/era5/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  app.launch_new_instance()


# Time and space mean

## Daily

In [12]:
def compute_spatialmean_pl(MR,varcode,where='above'):
    years = allyears
    name=MR.name
    months= MR.months
    spatialmeans=[]
    for y,m in itertools.product(years,months):
        ds = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name))
        varname = list(ds.data_vars)[0]
        ds=ds[varname]
        if where=='above':
            mask_era5= tilted_rect(ds,*MR.box_tilted,reverse=False)
        elif where=='upstream':
            mask_era5= tilted_rect(ds,*MR.box_tilted,reverse=True)
        spatialmeans.append(spatial_mean(resample_daily(ds),box=None,mask=mask_era5))
    temp=xr.concat(spatialmeans,dim='time')
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}-{}.{}.{}.{}mean.nc".format(varcode,years[0],years[-1],monthstr,name,where)
    temp.to_netcdf(filepath) 

In [ ]:
for MR in MRs:
    print(MR.name)
    %time compute_spatialmean_pl(MR,'128_130_t',where='upstream')
    %time compute_spatialmean_pl(MR,'128_133_q',where='upstream')

In [6]:
def concat_dailymeans_sfc(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 


def compute_upstream_spatialmean_sfc(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*MR.box_tilted,reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.sfc.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def compute_spatialmean_gpm(MR):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)) for m in months]
    mask_ghats_gpm= tilted_rect(var_months[0],*MR.box_tilted,reverse=False)
    var_months_sm  = [spatial_mean(var.precipitationCal,box=None,mask=mask_ghats_gpm) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{}.{}.abovemean.nc".format(years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def concat_dailymeans_gpm(MR):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "gpm_imerg_v06.{}-{}.{:02}.{}.zarr".format(years[0],years[-1],m,name)) for m in months]
    var_months  = [var.precipitationCal for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "gpm_imerg_v06.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 
    

In [7]:
for MR in MRs:
    print(MR.name)
    for varcode in ['228_246_100u','228_247_100v']:#,
        print(varcode)
        %time concat_dailymeans_sfc(MR,varcode)
    %time concat_dailymeans_gpm(MR)


cascades
228_246_100u
CPU times: user 12.7 s, sys: 1.1 s, total: 13.8 s
Wall time: 21.2 s
228_247_100v
CPU times: user 13.7 s, sys: 1.04 s, total: 14.7 s
Wall time: 22.5 s
CPU times: user 7.85 s, sys: 751 ms, total: 8.61 s
Wall time: 15.8 s


In [11]:
def compute_upstream_spatialmean_vinteg(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    mask_ghats_era5= tilted_rect(var_months[0],*MR.box_tilted,reverse=True)
    var_months_sm  = [spatial_mean(resample_daily(var[varname]),box=None,mask=mask_ghats_era5) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{}.{}.upstreammean.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months_sm,dim='time').sortby('time').to_netcdf(filepath) 

def concat_dailymeans_vinteg(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{:02}.{}.zarr".format(varcode,years[0],years[-1],m,name)) for m in months]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.vinteg.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').sortby('time').to_netcdf(filepath) 


In [12]:
for MR in MRs:
    print(MR)
    concat_dailymeans_vinteg(MR,'162_071_viwve')
    concat_dailymeans_vinteg(MR,'162_072_viwvn')

In [12]:
def compute_ebeLeLstar(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    ebs=[]
    eLs=[]
    eLstars=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        eb = 2.5e6/1004*q.sel(level=slice(lft_bot_pressure,1100))+t.sel(level=slice(lft_bot_pressure,1100))
        tL = t.sel(level=slice(lft_top_pressure,lft_bot_pressure-1))
        qL = q.sel(level=slice(lft_top_pressure,lft_bot_pressure-1))
        _,qs,_=humidsat(tL,tL.level)
        eL = 2.5e6/1004*qL+tL
        eLstar = 2.5e6/1004*qs+tL
        
        ebs.append(resample_daily(eb))
        eLs.append(resample_daily(eL))
        eLstars.append(resample_daily(eLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(ebs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLstars,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLstar{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [13]:
def compute_tLqL(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    tLs=[]
    qLs=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        tL = t.sel(level=slice(lft_top_pressure,lft_bot_pressure-1)).mean('level')
        qL = q.sel(level=slice(lft_top_pressure,lft_bot_pressure-1)).mean('level')
        tLs.append(resample_daily(tL))
        qLs.append(resample_daily(qL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(tLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.tL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(qLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.qL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [17]:
for MR in MRs:
    print(MR.name)
    #%time compute_ebeLeLstar(MR,'deep')
    #%time compute_ebeLeLstar(MR,'shallow')
    #%time compute_tLqL(MR,'deep')
    #%time compute_tLqL(MR,'shallow')
    %time compute_uLvL(MR)
    %time compute_u900v900(MR)

cascades
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 12
CPU times: user 39.6 s, sys: 4.36 s, total: 44 s
Wall time: 1min 32s
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 12
CP

In [15]:
def compute_uLvL(MR):
    years = allyears
    name=MR.name
    months= MR.months
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        u = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)).U
        v = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)).V
        uL = u.sel(level=slice(700.,899.)).mean('level')
        vL = v.sel(level=slice(700.,899.)).mean('level')
        uLs.append(resample_daily(uL))
        vLs.append(resample_daily(vL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(uLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.uL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(vLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.vL.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

In [16]:
def compute_u900v900(MR):
    years = allyears
    name=MR.name
    months= MR.months
    uLs=[]
    vLs=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        u = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_131_u',y,m,name)).U
        v = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_132_v',y,m,name)).V
        uL = u.sel(level = 900.)
        vL = v.sel(level = 900.)
        uLs.append(resample_daily(uL))
        vLs.append(resample_daily(vL))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(uLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.u900.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(vLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.v900.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    

# Additional

In [18]:
def compute_ebeLeLstardeep(MR):
    years = allyears
    name=MR.name
    months= MR.months
    ebs=[]
    eLs=[]
    eLstars=[]
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        eb = 2.5e6/1004*q.sel(level=slice(850,1100))+t.sel(level=slice(850,1100))
        tL = t.sel(level=slice(500.,849.))
        qL = q.sel(level=slice(500.,849.))
        _,qs,_=humidsat(tL,tL.level)
        eL = 2.5e6/1004*qL+tL
        eLstar = 2.5e6/1004*qs+tL
        
        ebs.append(resample_daily(eb))
        eLs.append(resample_daily(eL))
        eLstars.append(resample_daily(eLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    
    temp=xr.concat(ebs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.ebdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLdeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    temp=xr.concat(eLstars,dim='time')
    filepath = MAINDIR + "e5.diagnostic.eLstardeep.{}-{}.{}.{}.nc".format(years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
def thetaeb_vectorized(t,q,lft_bot_pressure=900.,pname='level'):
    t = t.sel({pname:slice(lft_bot_pressure,1100.)}) #900.
    q = q.sel({pname:slice(lft_bot_pressure,1100.)}) #900.
    es,qs,_=humidsat(t,t[pname])
    thetae =     thetae_perso(t,q,es,qs,pname=pname)
    return thetae.mean(pname)

def thetaeL_vectorized(t,q,lft_top_pressure,lft_bot_pressure,pname='level'):
    t = t.sel({pname:slice(lft_top_pressure,lft_bot_pressure-1)})
    q = q.sel({pname:slice(lft_top_pressure,lft_bot_pressure-1)})
    es,qs,_=humidsat(t,t[pname])
    thetae =     thetae_perso(t,q,es,qs,pname=pname)
    return thetae.mean(pname)

def thetaeLstar_vectorized(t,lft_top_pressure,lft_bot_pressure,pname='level'):
    t = t.sel({pname:slice(lft_top_pressure,lft_bot_pressure-1)})
    es,qs,_=humidsat(t,t[pname])
    thetaestar = thetaestar_perso(t,es,qs,pname=pname)  
    return thetaestar.mean(pname)

def compute_thetaebthetaeLthetaeLstar(MR,kind='shallow'):
    years = allyears
    name=MR.name
    months= MR.months
    thetaebs=[]
    thetaeLs=[]
    thetaeLstars=[]
    if kind=='shallow':
        lft_top_pressure=700.
        lft_bot_pressure=900.
        suffix=''
    elif kind=='deep':
        lft_top_pressure=500.
        lft_bot_pressure=850. 
        suffix='deep'
    
    for y,m in itertools.product(years,months):
        print(y,m)
        q = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_133_q',y,m,name)).Q
        t = xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format('128_130_t',y,m,name)).T
        #thetaeb = thetaeb_vectorized(t,q,lft_bot_pressure,pname='level')
        thetaeL = thetaeL_vectorized(t,q,lft_top_pressure,lft_bot_pressure,pname='level')
        #thetaeLstar = thetaeLstar_vectorized(t,lft_top_pressure,lft_bot_pressure,pname='level')
        #thetaebs.append(resample_daily(thetaeb).load())
        thetaeLs.append(resample_daily(thetaeL).load())
        #thetaeLstars.append(resample_daily(thetaeLstar))
    monthstr = '-'.join(["{:02}".format(m) for m in months])
        
    #temp=xr.concat(thetaebs,dim='time')
    #filepath = MAINDIR + "e5.diagnostic.thetaeb{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    #temp.to_netcdf(filepath) 
    
    temp=xr.concat(thetaeLs,dim='time')
    filepath = MAINDIR + "e5.diagnostic.thetaeL{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    temp.to_netcdf(filepath) 
    
    #temp=xr.concat(thetaeLstars,dim='time')
    #filepath = MAINDIR + "e5.diagnostic.thetaeLstar{}.{}-{}.{}.{}.nc".format(suffix,years[0],years[-1],monthstr,name)
    #temp.to_netcdf(filepath) 

In [19]:
for MR in MRs:
    print(MR.name)
    %time compute_thetaebthetaeLthetaeLstar(MR,'shallow')
    %time compute_thetaebthetaeLthetaeLstar(MR,'deep')

cascades
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 12
CPU times: user 1min 1s, sys: 5.16 s, total: 1min 6s
Wall time: 2min 35s
2001 1
2001 11
2001 12
2002 1
2002 11
2002 12
2003 1
2003 11
2003 12
2004 1
2004 11
2004 12
2005 1
2005 11
2005 12
2006 1
2006 11
2006 12
2007 1
2007 11
2007 12
2008 1
2008 11
2008 12
2009 1
2009 11
2009 12
2010 1
2010 11
2010 12
2011 1
2011 11
2011 12
2012 1
2012 11
2012 12
2013 1
2013 11
2013 12
2014 1
2014 11
2014 12
2015 1
2015 11
2015 12
2016 1
2016 11
2016 12
2017 1
2017 11
2017 12
2018 1
2018 11
2018 12
2019 1
2019 11
2019 12
2020 1
2020 11
2020 1

# Concatenate PLevs?

In [8]:
def concat_dailymeans_pl(MR,varcode):
    years=allyears
    name=MR.name
    months= MR.months
    var_months = [xr.open_zarr(MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}{:02}.{}.zarr".format(varcode,y,m,name)) for m in months for y in years]
    varname = list(var_months[0].data_vars)[0]
    var_months  = [resample_daily(var[varname]) for var in var_months]
    
    monthstr = '-'.join(["{:02}".format(m) for m in months])
    filepath = MAINDIR + "e5.oper.an.pl.{}.ll025sc.{}-{}.{}.{}.nc".format(varcode,years[0],years[-1],monthstr,name)
    xr.concat(var_months,dim='time').to_netcdf(filepath) 


In [9]:
for MR in MRs:
    print(MR.name)
    %time concat_dailymeans_pl(MR,'128_131_u')
    %time concat_dailymeans_pl(MR,'128_132_v')

ghats
CPU times: user 25 s, sys: 2.21 s, total: 27.2 s
Wall time: 1min 7s
CPU times: user 25.7 s, sys: 2.65 s, total: 28.4 s
Wall time: 1min 9s
myanmar
CPU times: user 24.6 s, sys: 2.38 s, total: 27 s
Wall time: 57.2 s
CPU times: user 25.6 s, sys: 2.46 s, total: 28.1 s
Wall time: 1min 7s
vietnam
CPU times: user 15.5 s, sys: 1.37 s, total: 16.9 s
Wall time: 38.8 s
CPU times: user 15.5 s, sys: 1.53 s, total: 17 s
Wall time: 37.4 s
malaysia
CPU times: user 15.9 s, sys: 1.45 s, total: 17.4 s
Wall time: 39.7 s
CPU times: user 15.8 s, sys: 1.57 s, total: 17.3 s
Wall time: 40.6 s
philippines
CPU times: user 16 s, sys: 1.49 s, total: 17.5 s
Wall time: 39.4 s
CPU times: user 15.4 s, sys: 1.45 s, total: 16.8 s
Wall time: 38.1 s
newbritain
CPU times: user 25.8 s, sys: 2.28 s, total: 28.1 s
Wall time: 1min 5s
CPU times: user 25.6 s, sys: 2.19 s, total: 27.7 s
Wall time: 1min 10s
